# Capacitated Vehicle Routing Problem

**Desenvolvedor**: Vítor Gabriel Reis Caitité 

In [1]:
# Imports
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from math import sqrt
import time 
from random import choice
import time
import random

## Função para cálculo da matrix de distâncias

In [2]:
# Cálculo da matrix de distâncias considerando a distância euclideana
def calculate_euc_distance_matrix(data):
    d = np.zeros((data.shape[0], data.shape[0])) # distance matrix
    for l in range(0, data.shape[0]):
        for c in range(0, data.shape[0]):
            d[l, c] = sqrt(sum((data[l] - data[c])**2))
    return d

## Nearest Neighbor

In [3]:
# Nearest neighbor for CVRP:
def nearest_neighbor(distance_matrix, demand, truck_capacity, depot = 0):
    unvisited = np.ones(distance_matrix.shape[0], dtype=bool) # to control which cities remain to visit   
    # The truck starts at the depot:
    path = [depot]    # variable to save the path
    unvisited[depot] = 0 
    cost = 0
    while np.sum(unvisited)>0:
        # First place to visit: 
        possible_starts = np.arange(0, distance_matrix.shape[0])
        start = choice(possible_starts[unvisited])
        path.append(start)  
        unvisited[start] = 0
        cost += distance_matrix[depot, path[-1]] 
        capacity = truck_capacity - demand[start]
        while capacity > 0 and np.sum(unvisited)>0:
            last_city = path[-1]
            # finding the nearest neighbor for the last city:
            next_idx = np.argmin(distance_matrix[last_city][unvisited]) 
            next_city = np.arange(distance_matrix.shape[0])[unvisited][next_idx] 
            # adding the nearest neighbor to the path if the capacity allows:
            if capacity >= demand[next_city]:
                path.append(next_city)
                unvisited[next_city] = 0
                capacity -= demand[next_city]
                # Updating the cost:
                cost += distance_matrix[last_city, next_city]
                if np.sum(unvisited)==0:
                    cost += distance_matrix[path[-1], path[0]]
                    path.append(depot)
                    break
            else:
                # restore the capacity:
                capacity = truck_capacity
                # add cost to return to depot:
                cost += distance_matrix[path[-1], path[0]]
                path.append(depot)
                break
    return path, cost

## Sweep Nearest Algorithm

In [4]:
# Cartesian coordinates to Polar:
# r = √( x^2 + y^2 )
# θ = tan-1 ( y / x )
def cart_to_pol(points, center):
    rho = np.sqrt((points[:,0]-center[0])**2 + (points[:,1]-center[1])**2)
    phi = np.arctan2((points[:,1]-center[1]), (points[:,0]-center[0]))
    return rho, (phi * 180/np.pi)%360

# Sweep Nearest Algorithm for CVRP:
def sweep_nearest(distance_matrix, demand, truck_capacity, theta, depot=0, first_idx=-1):
    sorted_idx = list(np.argsort(theta)) 
    unvisited = np.ones(distance_matrix.shape[0], dtype=bool) # to control which cities remain to visit   
    # The truck starts at the depot:
    path = [depot]    # variable to save the path
    unvisited[depot] = 0 
    cost = 0
    # initial idx (the angle of first place chosen will be the initial angle to start the sweep)
    idx = first_idx
    # random initial idx (it can be understood as choose a random angle to start the sweep)
    if idx < 0 or idx > len(theta)-1:
        idx = random.randint(0, len(theta)-1)
    while np.sum(unvisited)>0:
        # First place to visit is chosen by the sweep algorithm:
        first_place = sorted_idx[idx] 
        idx = (idx + 1) % len(theta)
        # Checking if a possible initial place was visited: 
        if unvisited[first_place] == False:
            continue
        else: 
            path.append(first_place)  
            unvisited[first_place] = 0
            cost += distance_matrix[depot, path[-1]] 
            capacity = truck_capacity - demand[first_place]
            while capacity > 0 and np.sum(unvisited)>0:
                last_city = path[-1]
                # finding the nearest neighbor for the last city:
                next_idx = np.argmin(distance_matrix[last_city][unvisited]) 
                next_city = np.arange(distance_matrix.shape[0])[unvisited][next_idx] 
                # adding the nearest neighbor to the path if the capacity allows:
                if capacity >= demand[next_city]:
                    path.append(next_city)
                    unvisited[next_city] = 0
                    capacity -= demand[next_city]
                    # Updating the cost:
                    cost += distance_matrix[last_city, next_city]
                else:
                    # restore the capacity:
                    capacity = truck_capacity
                    break
            # add cost to return to depot:      
            cost += distance_matrix[path[-1], path[0]]
            path.append(depot)
    return path, cost

## Sweep + Nearest Insertion Algorithm for CVRP

In [5]:
# Sweep Nearest Insertion Algorithm for CVRP:
def sweep_nearest_insertion(distance_matrix, demand, truck_capacity, theta, depot=0, first_idx=-1):
    sorted_idx = list(np.argsort(theta)) 
    unvisited = np.ones(distance_matrix.shape[0], dtype=bool) # to control which cities remain to visit   
    # The truck starts at the depot:
    path = []    # variable to save the entire path
    unvisited[depot] = 0 
    cost = 0
    # initial idx (the angle of first place chosen will be the initial angle to start the sweep)
    idx = first_idx
    # random initial idx (it can be understood as choose a random angle to start the sweep)
    if idx < 0 or idx > len(theta)-1:
        idx = random.randint(0, len(theta)-1)
    while np.sum(unvisited)>0:
        # First place to visit is chosen by the sweep algorithm:
        first_place = sorted_idx[idx] 
        idx = (idx + 1) % len(theta)
        # Checking if a possible initial place was visited: 
        if unvisited[first_place] == False:
            continue
        else: 
            tour = [depot, first_place] # variable to save the tour until we get back to the depot
            unvisited[first_place] = 0
            capacity = truck_capacity - demand[first_place]
            if np.sum(unvisited)==0:
                tour.append(depot)
            while capacity > 0 and np.sum(unvisited)>0:
                smaller = np.inf
                # (Selection step) Given a sub-tour, find node r not 
                # in the sub-tour closest to any node j in the sub-tour; i.e. with minimal c_rj 
                for i in range(len(tour)):                
                    idx = np.argmin(distance_matrix[tour[i]][unvisited])
                    min_distance = distance_matrix[tour[i]][unvisited][idx]
                    if min_distance < smaller:
                        smaller_idx = np.arange(distance_matrix.shape[0])[unvisited][idx]
                        smaller = min_distance
                # adding the city to the path if the capacity allows:
                if capacity >= demand[smaller_idx]:
                    unvisited[smaller_idx] = 0 
                    capacity -= demand[smaller_idx]
                    insertion_cost = np.inf
                    tour.append(tour[0])
                    # (Insertion step) Find the arc (i, j) in the sub-tour
                    # which minimizes c_ir + c_rj - c_ij . Insert r between i and j. 
                    for i in range(len(tour)-1):
                        new_insertion_cost = distance_matrix[tour[i], smaller_idx] + distance_matrix[tour[i+1], smaller_idx] - distance_matrix[tour[i], tour[i+1]]  
                        if new_insertion_cost < insertion_cost:
                            insertion_cost = new_insertion_cost
                            insertion_idx = i
                    tour.insert(insertion_idx+1, smaller_idx)
                    tour = tour[:-1]
                    if np.sum(unvisited)==0:
                        tour.append(depot)
                        break
                else:
                    break
            # restore the capacity:
            capacity = truck_capacity
            path += tour
    # Calculating the cost:
    cost = 0 
    for i in range(len(path)-1):
        cost += distance_matrix[path[i], path[i+1]]
    cost += distance_matrix[path[-1], path[0]]
    return path, cost

## Sweep + Farthest Insertion for CVRP

In [6]:
# Sweep Farthest Algorithm for CVRP:
def sweep_farthest_insertion(distance_matrix, demand, truck_capacity, theta, depot=0, first_idx=-1):
    sorted_idx = list(np.argsort(theta)) 
    unvisited = np.ones(distance_matrix.shape[0], dtype=bool) # to control which cities remain to visit   
    # The truck starts at the depot:
    path = []    # variable to save the entire path
    unvisited[depot] = 0 
    # initial idx (the angle of first place chosen will be the initial angle to start the sweep)
    idx = first_idx
    # random initial idx (it can be understood as choose a random angle to start the sweep)
    if idx < 0 or idx > len(theta)-1:
        idx = random.randint(0, len(theta)-1)
    while np.sum(unvisited)>0:
        # First place to visit is chosen by the sweep algorithm:
        first_place = sorted_idx[idx] 
        idx = (idx + 1) % len(theta)
        # Checking if a possible initial place was visited: 
        if unvisited[first_place] == False:
            continue
        else: 
            tour = [depot, first_place] # variable to save the tour until we get back to the depot
            unvisited[first_place] = 0
            capacity = truck_capacity - demand[first_place]
            if np.sum(unvisited)==0:
                tour.append(depot)
            while capacity > 0 and np.sum(unvisited)>0:
                tour_distances = {}
                # (Selection step) Given a sub-tour, find a
                # node not in the sub-tour farthest to the sub-tour;
                for i in range(distance_matrix.shape[0]):
                    if not unvisited[i]:
                        continue
                    distances = []
                    for j in range(len(tour)):  
                        distances.append(distance_matrix[tour[j], i])
                    tour_distances[i] = min(distances) 
                larger_idx = max(tour_distances, key=tour_distances.get)
                
                # adding the city to the path if the capacity allows:
                if capacity >= demand[larger_idx]:
                    unvisited[larger_idx] = 0
                    insertion_cost = np.inf
                    capacity -= demand[larger_idx]
                    tour.append(tour[0])
                    # (Insertion step) Find the arc (i, j) in the sub-tour
                    # which minimizes c_ir + c_rj - c_ij . Insert r between i and j. 
                    for i in range(len(tour)-1):
                        new_insertion_cost = distance_matrix[tour[i], larger_idx] + distance_matrix[larger_idx, tour[i+1]] - distance_matrix[tour[i], tour[i+1]]  
                        if new_insertion_cost < insertion_cost:
                            insertion_cost = new_insertion_cost
                            insertion_idx = i
                    tour.insert(insertion_idx+1, larger_idx)
                    tour = tour[:-1]
                    if np.sum(unvisited)==0:
                        tour.append(depot)
                else:
                    break
            capacity = truck_capacity
            path += tour
    # Calculating the cost:
    cost = 0 
    for i in range(len(path)-1):
        cost += distance_matrix[path[i], path[i+1]]
    cost += distance_matrix[path[-1], path[0]]
    return path, cost

## Sweep + Cheapest Insertion for CVRP

In [7]:
# Sweep Cheapest  Algorithm for CVRP:
def sweep_cheapest_insertion(distance_matrix, demand, truck_capacity, theta, depot=0, first_idx=-1):
    sorted_idx = list(np.argsort(theta)) 
    unvisited = np.ones(distance_matrix.shape[0], dtype=bool) # to control which cities remain to visit   
    # The truck starts at the depot:
    path = []    # variable to save the entire path
    unvisited[depot] = 0 
    cost = 0
    # initial idx (the angle of first place chosen will be the initial angle to start the sweep)
    idx = first_idx
    new_tour = []
    # random initial idx (it can be understood as choose a random angle to start the sweep)
    if idx < 0 or idx > len(theta)-1:
        idx = random.randint(0, len(theta)-1)
    while np.sum(unvisited)>0:
        # First place to visit is chosen by the sweep algorithm:
        first_place = sorted_idx[idx] 
        idx = (idx + 1) % len(theta)
        # Checking if a possible initial place was visited: 
        if unvisited[first_place] == False:
            continue
        else: 
            tour = [depot, first_place] # variable to save the tour until we get back to the depot
            unvisited[first_place] = 0
            capacity = truck_capacity - demand[first_place]
            if np.sum(unvisited)==0:
                tour.append(depot)
            while capacity > 0 and np.sum(unvisited)>0:
                # Find the closest neighbor to the tour (min cik + ckj - cij)
                smaller = np.inf
                tour.append(tour[0])
                for k in range(distance_matrix.shape[0]):
                    if not unvisited[k]:
                        continue
                    # Finding the city which the cost of insert (dik + dkj − dij ) is the cheapest one.
                    for i in range(len(tour)-1):
                        d = distance_matrix[tour[i]][k] + distance_matrix[k][tour[i+1]] - distance_matrix[tour[i]][tour[i+1]]
                        if d < smaller:
                            smaller = d
                            new_tour = tour[:i + 1] + [k] + tour[i + 1:-1]
                            new_city = k
                tour = tour[:-1]
                 # adding the city to the path if the capacity allows:
                if capacity >= demand[new_city]:
                    unvisited[new_city] = 0
                    tour = new_tour
                    # Updating the cost
                    cost += smaller
                    capacity -= demand[new_city]
                    if np.sum(unvisited)==0:
                        tour.append(depot)
                else:
                    break
            capacity = truck_capacity
            path += tour
    # Calculating the cost:
    cost = 0 
    for i in range(len(path)-1):
        cost += distance_matrix[path[i], path[i+1]]
    cost += distance_matrix[path[-1], path[0]]
    return path, cost

## Função para aplicação dos testes

In [8]:
def apply_test(arquives, n_tests):
    final_results = [["Arquivo", 'Nearest Neighbor', 'Sweep Nearest', 'Sweep Cheapest', \
                      'Sweep Nearest Ins', 'Sweep Farthest Ins']]
    final_time_results = [["Arquivo", 'Nearest Neighbor', 'Sweep Nearest', 'Sweep Cheapest', \
                          'Sweep Nearest Ins', 'Sweep Farthest Ins']]
    path = '/home/vitor/Documents/UFMG/Mastering/3/Heurística_Meta-heurística/TPs/heuristics-and-metaheuristics/TP_Final _Article_VRPC/instances/to_test/'
    for arquive in arquives:
        df = pd.read_csv(f'{path}{arquive}', skiprows=range(0, 4), sep='\s+', names=["i","X", "Y"]) 
        if df["Y"].iloc[0] == "EUC_2D":
            df = df[:-4]
            truck_capacity = float(df["Y"].iloc[1])
            df1 = df[(df.index[df.i == 'DEMAND_SECTION'][0] + 1):df.shape[0]]
            demand = df1['X'].to_numpy(dtype=float)
            coord = df[3:(df.index[df.i == 'DEMAND_SECTION'][0])][["X", "Y"]].to_numpy(dtype=float)
            _, theta = cart_to_pol(coord, center=coord[0,:])
        else:
            continue       
        distance_matrix = calculate_euc_distance_matrix(coord)
        nn_results = np.zeros(n_tests) # for nearest neighbor
        nn_time_results = np.zeros(n_tests) # for nearest neighbor
        sn_results = np.zeros(n_tests) # for sweep nearest
        sn_time_results = np.zeros(n_tests) # for sweep nearest
        sci_results = np.zeros(n_tests) # for sweep cheapest insertion
        sci_time_results = np.zeros(n_tests) # for sweep cheapest insertion
        sni_results = np.zeros(n_tests) # for sweep nearest insertion
        sni_time_results = np.zeros(n_tests) # for sweep nearest insertion
        sfi_results = np.zeros(n_tests) # for sweep farthest insertion
        sfi_time_results = np.zeros(n_tests) # for sweep farthest insertion
        
        for idx in range(n_tests):
            # Nearest Neighbor:
            start_time = time.time()
            _, nn_results[idx] = nearest_neighbor(distance_matrix, demand, truck_capacity=truck_capacity, depot=0)
            end_time = time.time()
            nn_time_results[idx] = (end_time - start_time)*1000
            # Sweep Nearest:
            start_time = time.time()
            _, sn_results[idx] = sweep_nearest(distance_matrix, demand, truck_capacity=truck_capacity, theta=theta, depot=0)
            end_time = time.time()
            sn_time_results[idx] = (end_time - start_time)*1000
            # Sweep + Cheapest Insertion
            start_time = time.time()
            _, sci_results[idx] = sweep_cheapest_insertion(distance_matrix, demand, truck_capacity=truck_capacity, theta=theta, depot=0)
            end_time = time.time()
            sci_time_results[idx] = (end_time - start_time)*1000
            # Sweep + Nearest Insertion
            start_time = time.time()
            _, sni_results[idx] = sweep_nearest_insertion(distance_matrix, demand, truck_capacity=truck_capacity, theta=theta, depot=0)
            end_time = time.time()
            sni_time_results[idx] = (end_time - start_time)*1000
            # Sweep + Farthest Insertion
            start_time = time.time()
            _, sfi_results[idx] = sweep_farthest_insertion(distance_matrix, demand, truck_capacity=truck_capacity, theta=theta, depot=0)
            end_time = time.time()
            sfi_time_results[idx] = (end_time - start_time)*1000 
            
        result = [arquive,
                 '{:.0f}'.format(nn_results.mean()) + " +/- " + '{:.0f}'.format(nn_results.std()),
                 '{:.0f}'.format(sn_results.mean()) + " +/- " + '{:.0f}'.format(sn_results.std()),
                 '{:.0f}'.format(sci_results.mean()) + " +/- " + '{:.0f}'.format(sci_results.std()),
                 '{:.0f}'.format(sni_results.mean()) + " +/- " + '{:.0f}'.format(sni_results.std()),
                 '{:.0f}'.format(sfi_results.mean()) + " +/- " + '{:.0f}'.format(sfi_results.std())]
        
        time_result = [arquive,
                      '{:.1f}'.format(nn_time_results.mean()) + " +/- " + '{:.1f}'.format(nn_time_results.std()),
                      '{:.1f}'.format(sn_time_results.mean()) + " +/- " + '{:.1f}'.format(sn_time_results.std()),
                      '{:.1f}'.format(sci_time_results.mean()) + " +/- " + '{:.1f}'.format(sci_time_results.std()),
                      '{:.1f}'.format(sni_time_results.mean()) + " +/- " + '{:.1f}'.format(sni_time_results.std()),
                      '{:.1f}'.format(sfi_time_results.mean()) + " +/- " + '{:.1f}'.format(sfi_time_results.std())]    
        final_results.append(result)
        final_time_results.append(time_result)
    return final_results, final_time_results

## Resultados

In [9]:
to_test = ["A-n32-k5.vrp",
"A-n37-k5.vrp",
"A-n39-k6.vrp",
"A-n48-k7.vrp",
"A-n63-k10.vrp",
"B-n39-k5.vrp",
"B-n51-k7.vrp",
"B-n67-k10.vrp",
"B-n78-k10.vrp",
"E-n101-k14.vrp",
"E-n51-k5.vrp",
"E-n76-k10.vrp",
"F-n45-k4.vrp",
"F-n72-k4.vrp",
"F-n135-k7.vrp",
"M-n101-k10.vrp",
"M-n121-k7.vrp",
"M-n151-k12.vrp",
"M-n200-k16.vrp",
'P-n40-k5.vrp',
'P-n45-k5.vrp',
'P-n70-k10.vrp',
'P-n76-k5.vrp',
'X-n120-k6.vrp',
'X-n200-k36.vrp',
'X-n401-k29.vrp',
'X-n599-k92.vrp',
'X-n895-k37.vrp',
'X-n957-k87.vrp',
'X-n1001-k43.vrp']

In [10]:
import os
import re
import tabulate
arquives = []
path = '/home/vitor/Documents/UFMG/Mastering/3/Heurística_Meta-heurística/TPs/heuristics-and-metaheuristics/TP_Final _Article_VRPC/instances/to_test'
for root, dirs, files in os.walk(path):
    for filename in sorted(files):
        if re.search(r'.vrp', filename) and filename in to_test:
            arquives.append(filename)
results, time_results = apply_test(arquives, 50)
# Generating table results:
table1 = tabulate.tabulate(results, tablefmt='grid')
table2 = tabulate.tabulate(time_results, tablefmt='grid')

### Custo obtido por cada algoritmo em cada arquivo de teste.

In [11]:
print(table1)

+-----------------+------------------+----------------+-----------------+-------------------+--------------------+
| Arquivo         | Nearest Neighbor | Sweep Nearest  | Sweep Cheapest  | Sweep Nearest Ins | Sweep Farthest Ins |
+-----------------+------------------+----------------+-----------------+-------------------+--------------------+
| A-n32-k5.vrp    | 998 +/- 73       | 1005 +/- 42    | 1013 +/- 62     | 991 +/- 75        | 1383 +/- 67        |
+-----------------+------------------+----------------+-----------------+-------------------+--------------------+
| A-n37-k5.vrp    | 906 +/- 52       | 894 +/- 28     | 864 +/- 31      | 975 +/- 53        | 1184 +/- 41        |
+-----------------+------------------+----------------+-----------------+-------------------+--------------------+
| A-n39-k6.vrp    | 988 +/- 62       | 1031 +/- 41    | 1118 +/- 93     | 1141 +/- 86       | 1438 +/- 58        |
+-----------------+------------------+----------------+-----------------+-------

### Tempo de execução (ms) de cada algoritmo em cada arquivo de teste. Processador i7 7th Gen., 8 Gb de RAM, sistema Linux.

In [12]:
print(table2)

+-----------------+------------------+---------------+------------------+-------------------+--------------------+
| Arquivo         | Nearest Neighbor | Sweep Nearest | Sweep Cheapest   | Sweep Nearest Ins | Sweep Farthest Ins |
+-----------------+------------------+---------------+------------------+-------------------+--------------------+
| A-n32-k5.vrp    | 0.6 +/- 0.3      | 0.6 +/- 0.3   | 2.6 +/- 0.7      | 1.5 +/- 0.5       | 1.3 +/- 0.3        |
+-----------------+------------------+---------------+------------------+-------------------+--------------------+
| A-n37-k5.vrp    | 0.6 +/- 0.2      | 0.6 +/- 0.2   | 3.6 +/- 0.6      | 1.7 +/- 0.4       | 1.6 +/- 0.3        |
+-----------------+------------------+---------------+------------------+-------------------+--------------------+
| A-n39-k6.vrp    | 0.6 +/- 0.2      | 0.6 +/- 0.1   | 3.5 +/- 0.6      | 1.7 +/- 0.5       | 1.8 +/- 0.3        |
+-----------------+------------------+---------------+------------------+-------